In [1]:
import polars as pl
import pandas as pd
import matplotlib.pyplot as plt

import codecs
import os

import matplotlib.font_manager as fm

# 한글 글꼴 경로 설정 (예: Malgun Gothic)
font_path = "C:/Windows/Fonts/malgun.ttf"  # Windows의 경우
font_prop = fm.FontProperties(fname=font_path, size=12)

# Matplotlib의 기본 글꼴 설정
plt.rc('font', family=font_prop.get_name())

In [7]:

# CSV 파일 경로
main_path = 'C:/Users/campus3S031/Desktop/iM뱅크 프로젝트용 데이터/raw_data/'

# 위에꺼 사실상 무시 가능
ori_path = [
    f"{main_path}iMBANK_CUSTOMER_DATA_2021(K-DigitalTraining).csv",
    f"{main_path}iMBANK_CUSTOMER_DATA_2022(K-DigitalTraining).csv",
    f"{main_path}iMBANK_CUSTOMER_DATA_2023(K-DigitalTraining).csv",
    f"{main_path}iMBANK_CARD_DATA_2021(K-DigitalTraining).csv",
    f"{main_path}iMBANK_CARD_DATA_2022(K-DigitalTraining).csv",
    f"{main_path}iMBANK_CARD_DATA_2023(K-DigitalTraining).csv"
]

tmp_path = [
    "data/cus_utf8_file_2021.csv",
    "data/cus_utf8_file_2022.csv",
    "data/cus_utf8_file_2023.csv",
    "data/card_utf8_file_2021.csv",
    "data/card_utf8_file_2022.csv",
    "data/card_utf8_file_2023.csv"
]


In [3]:
# 원본 파일을 읽어서 UTF-8로 변환하여 임시 파일에 저장 -> 한 번만 실행해도 충분
# 메모리 에러 발생
'''
for i in range(6):
    with codecs.open(ori_path[i], 'r', encoding='cp949') as f_in:
        content = f_in.read()
    with codecs.open(tmp_path[i], 'w', encoding='utf8') as f_out:
        f_out.write(content)
'''

"\nfor i in range(6):\n    with codecs.open(ori_path[i], 'r', encoding='cp949') as f_in:\n        content = f_in.read()\n    with codecs.open(tmp_path[i], 'w', encoding='utf8') as f_out:\n        f_out.write(content)\n"

# 손님 데이터 수정


In [4]:
# LazyFrame 생성
new_path = "C:/Users/campus3S031/Desktop/iMBank/iM_DiGital_Banker_academy/projects/iMBank_pj5/data/"

cus_path = [
    "cus_utf8_file_2021.csv",
    "cus_utf8_file_2022.csv",
    "cus_utf8_file_2023.csv"
]

card_path = [
    "card_utf8_file_2021.csv",
    "card_utf8_file_2022.csv",
    "card_utf8_file_2023.csv"
]


In [5]:

def join_customer_and_card_data(file_list_1, file_list_2):
    
    processed_data = []

    for i in range(len(file_list_1)):
        lazy_df = pl.scan_csv(f'{new_path}{file_list_1[i]}', encoding='utf8')
        lazy_df_2 = pl.scan_csv(f'{new_path}{file_list_2[i]}', encoding='utf8')

        # 특정 열만 선택
        cus_df = lazy_df # 기존 고객 데이터

        card_df = lazy_df_2.group_by('고객ID').agg([
            pl.col('승인금액').sum().alias("카드이용 금액"),
            pl.col('고객ID').count().alias("카드이용 횟수"),
        ])

        # left join 수행
        joined_df = cus_df.join(
            card_df,
            left_on='고객ID',  
            right_on='고객ID',      
            how='left'
        )
        processed_data.append(joined_df)
    
    return processed_data

processed_data = join_customer_and_card_data(cus_path, card_path)

In [6]:
# 실제 데이터 불러오기 (lazy 실행)
joined_df_collected_1 = processed_data[0].collect()
joined_df_collected_2 = processed_data[1].collect()
joined_df_collected_3 = processed_data[2].collect()

# 데이터 미리보기
print("\n한글 인코딩 데이터 미리보기:")
print(joined_df_collected_1.filter(
    pl.col('카드이용 횟수').is_not_null()
).head())

# 데이터 정보 확인
print("\n데이터 정보:")
print(f"행 수: {joined_df_collected_1.height}")
print(f"열 수: {joined_df_collected_1.width}")
print(f"컬럼명: {joined_df_collected_1.columns}")


한글 인코딩 데이터 미리보기:
shape: (5, 15)
┌──────────┬────────────┬──────────┬──────┬───┬───────────────┬──────────┬──────────┬──────────┐
│ 기준년월 ┆ 고객ID     ┆ 연령대   ┆ 성별 ┆ … ┆ 수신_외화예금 ┆ 대출금액 ┆ 카드이용 ┆ 카드이용 │
│ ---      ┆ ---        ┆ ---      ┆ ---  ┆   ┆ ---           ┆ ---      ┆ 금액     ┆ 횟수     │
│ i64      ┆ i64        ┆ str      ┆ str  ┆   ┆ i64           ┆ i64      ┆ ---      ┆ ---      │
│          ┆            ┆          ┆      ┆   ┆               ┆          ┆ i64      ┆ u32      │
╞══════════╪════════════╪══════════╪══════╪═══╪═══════════════╪══════════╪══════════╪══════════╡
│ 202112   ┆ 9999998161 ┆ 60대이상 ┆ 여   ┆ … ┆ 0             ┆ 0        ┆ 1060000  ┆ 40       │
│ 202112   ┆ 9999981183 ┆ 60대이상 ┆ 여   ┆ … ┆ 0             ┆ 0        ┆ 5430000  ┆ 120      │
│ 202112   ┆ 9999972286 ┆ 60대이상 ┆ 여   ┆ … ┆ 0             ┆ 0        ┆ 10000    ┆ 1        │
│ 202112   ┆ 9999933174 ┆ 60대이상 ┆ 여   ┆ … ┆ 0             ┆ 0        ┆ 520000   ┆ 14       │
│ 202112   ┆ 9999918728 ┆ 60대이상 ┆ 여   ┆ … ┆ 0 

# 비지도 기법
- 신용점수 도출 
- 메모리 부족족

In [15]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans


# LazyFrame을 DataFrame으로 변환
df = processed_data[0].collect().to_pandas()

# 클러스터링을 위한 특성 선택 및 전처리
features = ['수신_요구불예금', '수신_거치식예금', '수신_적립식예금', '수신_펀드', '수신_외화예금', '대출금액', "대출금액", "카드이용 금액", "카드이용 횟수"]
scaler = StandardScaler()
scaled_features = scaler.fit_transform(df[features])

# AutoEncoder 모델 정의
class AutoEncoder(nn.Module):
    def __init__(self, input_dim, encoding_dim=2):
        super(AutoEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 8),
            nn.ReLU(),
            nn.Linear(8, encoding_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, 8),
            nn.ReLU(),
            nn.Linear(8, input_dim)
        )
    
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

# 모델 초기화
torch.manual_seed(42)
input_dim = scaled_features.shape[1]
autoencoder = AutoEncoder(input_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=0.01)

# Tensor 변환
data_tensor = torch.FloatTensor(scaled_features)

# 학습 과정
epochs = 100
for epoch in range(epochs):
    optimizer.zero_grad()
    encoded, decoded = autoencoder(data_tensor)
    loss = criterion(decoded, data_tensor)
    loss.backward()
    optimizer.step()

# 인코딩된 차원 축소 데이터 추출
encoded_data, _ = autoencoder(data_tensor)
encoded_data = encoded_data.detach().numpy()

# K-Means 클러스터링 수행
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
df["신용점수"] = kmeans.fit_predict(encoded_data)

# 클러스터를 점수로 변환 (가중치 적용 가능)
df["신용점수"] = df["신용점수"].map({
    0: 600,
    1: 750,
    2: 850
})

# Polars DataFrame으로 변환
result_df = pl.from_pandas(df)
print(result_df)



MemoryError: Unable to allocate 1.40 GiB for an array with shape (20835992, 9) and data type float64